# Jingyuan Zhou
# Assignment 2: Text Classification
# Implementation/Experimentation with Linear Text Classifiers

In [196]:
import pandas as pd
import numpy as np
import string
from scipy.sparse import lil_matrix, coo_matrix, csr_matrix
# from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

ImportError: No module named nltk.corpus

In [197]:
import nltk

ImportError: No module named nltk

In [2]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [158]:
train = pd.read_table('sst3.train', header = None)
train.columns = ['text', 'label']

dev = pd.read_table('sst3.dev', header = None)
dev.columns = ['text', 'label']

devtest = pd.read_table('sst3.devtest', header = None)
devtest.columns = ['text', 'label']

In [6]:
def process(t):
#     l = t.lower().translate(string.maketrans("",""), string.punctuation).strip().split()
#     temp = [WordNetLemmatizer().lemmatize(x) for x in l if x != '' and x not in stopwords]
    l = t.strip().split()
    return l

def process_df(df):
    df['tokens'] = df['text'].apply(lambda x: process(x), 1)
    df.reset_index(inplace = True)
    #return df

## 1.1 Building a Linear Classifier

In [159]:
process_df(train)
process_df(dev)
process_df(devtest)

In [8]:
train.head(3)

,index,text,label,tokens
0,0,The Rock is destined to be the 21st Century 's...,2,"[The, Rock, is, destined, to, be, the, 21st, C..."
1,1,is destined to be the 21st Century 's new `` C...,2,"[is, destined, to, be, the, 21st, Century, 's,..."
2,2,is destined to be the 21st Century 's new `` C...,2,"[is, destined, to, be, the, 21st, Century, 's,..."


In [9]:
vocab = list(set(sum(list(train['tokens']), [])))

# create feature functions based on training set
feature_index_dic = {}
ind = 0
ind_0 = []
ind_1 = []
ind_2 = []
for word in vocab:
    for i in range(3):
        feature_index_dic[(word, i)] = ind
        if i == 0:
            ind_0.append(ind)
        elif i == 1:
            ind_1.append(ind)
        else:
            ind_2.append(ind)
        ind += 1

label = [a[1] for a in feature_index_dic.keys()] #for train data set

In [10]:
len(vocab)

18106

In [15]:
def populate_feature(l, i_list, j_list, v_list, istrain = True):
    i = l['index']
    words = l['tokens']
    if istrain:
        label = l['label']
        for w in words:
            j = feature_index_dic[(w, label)]
            i_list.append(i)
            j_list.append(j)
            v_list.append(1)
    else:
        for w in words:
            for label in (0, 1, 2):
                j = feature_index_dic[(w, label)]
                i_list.append(i)
                j_list.append(j)
                v_list.append(1)               
            
#             l[(w, label)] = 1
#             print 'Processed ('+str(w)+str(label)+')'

def vectorize(df, istrain = True):
    i_list = []
    j_list = []
    v_list = []
    discard = df.apply(lambda x: populate_feature(x, i_list, j_list, v_list, istrain), 1)
    f_M = coo_matrix((v_list, (i_list, j_list)), shape=(len(train), len(feature_index_dic)))
    return f_M.tocsr()

def classify(f_v, w):
    v_0 = np.dot(f_v[ind_0], w[ind_0])
    v_1 = np.dot(f_v[ind_1], w[ind_1])
    v_2 = np.dot(f_v[ind_2], w[ind_2])
#     print v
    return np.argmax([v_0, v_1, v_2])

In [12]:
f_M_train = vectorize(train)
#f_M_test = vectorize(test, istrain = False)
#f_M_dev = vectorize(dev, istrain = False)

In [19]:
w = np.zeros(len(feature_index_dic))#.reshape(len(feature_index_dic),1) # for each epoch
for i in range(len(train)):
    f_v = f_M_train.getrow(i)
    c = classify(f_v.toarray()[0], w)
    y_2 = [int(l == c) for l in label]
    w = w + 0.01*f_v - 0.01* np.array(y_2)
    if i%20000 == 0:
        print 'finished'+str(i)+'entries'

IndexError: index (54315) out of range

In [ ]:
def test(f_M):
    

# Another way

In [114]:
# at first, get all the features
def populate_feature(l, features):
    i = l['index']
    words = l['tokens']
    label = l['label']
    for w in words:
        if (w, label) not in features:
            features.append((w, label))
               
features = []
discard = train.apply(lambda x: populate_feature(x, features), 1)

In [115]:
len(features)

36964

In [117]:
def score (word, y, f_weight_dic):
    try:
        return f_weight_dic[(word, y)]
    except:
        return 0

In [118]:
def classify_p(tokens, f_weight_dic):
    scores = []
    for y in range(2):
        s = 0
        for word in tokens:
            s += score(word, y, f_weight_dic)
        scores.append(s)
    return np.argmax(scores)

In [171]:
def classify_h(tokens, label, f_weight_dic):
    scores = []
    for y in range(2):
        s = 0
        for word in tokens:
            s += score(word, y, f_weight_dic) + (y != label)   
        scores.append(s)
    return np.argmax(scores)

In [153]:
def test(df, f_weigt_dic):
    acc = 0
    for i in range(len(df)):
        line = df.iloc[i,:]
        label = line['label']
        tokens = list(set(line['tokens']))
        acc += (label==classify(tokens, f_weight_dic))
#     print acc
    r = acc*100/float(len(df))
    print format(r, '.2f')+'%'
    return r
#     return format(acc*100/len(dev), '.2f')+'%'

In [165]:
def perceptron_loss(word, label, f_weight_dic):
    for y in range(3):
        if y == label:
            f_weight_dic[(word, y)] = f_weight_dic[(word, y)] + 0.01-\
                            0.01*(label == classify_p(tokens, f_weight_dic))
        else:
            try:
                f_weight_dic[(word, y)] = f_weight_dic[(word, y)]-\
                                    0.01*(label == classify_p(tokens, f_weight_dic))
            except:
                continue
    return f_weight_dic

In [174]:
def hinge_loss(word, label, f_weight_dic):
    for y in range(3):
        if y == label:
            f_weight_dic[(word, y)] = f_weight_dic[(word, y)] + 0.01-\
                            0.01*(label == classify_h(tokens, label, f_weight_dic))
        else:
            try:
                f_weight_dic[(word, y)] = f_weight_dic[(word, y)]-\
                                    0.01*(label == classify_(tokens, label, f_weight_dic))
            except:
                continue  
    return f_weight_dic

In [190]:
def ssd(loss, data = train):
    best_score = 0
    f_weight_dic = dict.fromkeys(features, 0)


    for epoch in range(1, 21):
        for i in range(len(train)):
            line = train.iloc[i,:]
            label = line['label']
            tokens = set(line['tokens'])
            for word in tokens:
                if loss == 'perceptron':
                    f_weight_dic = perceptron_loss(word, label, f_weight_dic)
                
                else:
                    f_weight_dic = hinge_loss(word, label, f_weight_dic)
    #         if i%20000 == 0 and i!= 0:
    #             print 'Epoch %d position %d ' % (epoch, i)
    #             res = test(dev, f_weight_dic)
    #             if res > best_score:
    #                 best_score = res
    #                 best_w = f_weight_dic
            break
        break
        print 'End of epoch '+ str(epoch)
        res = test(dev, f_weight_dic)
        break
    #     if res > best_score:
    #         best_score = res
    #         best_w = f_weight_dic
#     test(devtest, f_weight_dic
    return f_weight_dic

In [191]:
f_weight_dic = ssd('perceptron')

In [176]:
ssd('h')

End of epoch 1
39.82%
37.93%


In [124]:
line = dev.iloc[0,:]
label = line['label']
tokens = line['tokens']

classify(tokens, f_weight_dic)

In [192]:
tokens = train.iloc[0, :]['tokens']
label = train.iloc[0, :]['label']

In [193]:
t = zip(tokens, [label]*len(tokens))
for i in t:
    print i
    print f_weight_dic[i]

('The', 2)
0.01
('Rock', 2)
0.01
('is', 2)
0.01
('destined', 2)
0.01
('to', 2)
0.01
('be', 2)
0.01
('the', 2)
0.01
('21st', 2)
0.01
('Century', 2)
0.01
("'s", 2)
0.01
('new', 2)
0.01
('``', 2)
0.01
('Conan', 2)
0.01
("''", 2)
0.01
('and', 2)
0.01
('that', 2)
0.01
('he', 2)
0.01
("'s", 2)
0.01
('going', 2)
0.01
('to', 2)
0.01
('make', 2)
0.01
('a', 2)
0.01
('splash', 2)
0.01
('even', 2)
0.01
('greater', 2)
0.01
('than', 2)
0.01
('Arnold', 2)
0.01
('Schwarzenegger', 2)
0.01
(',', 2)
0.01
('Jean-Claud', 2)
0.01
('Van', 2)
0.01
('Damme', 2)
0.01
('or', 2)
0.01
('Steven', 2)
0.01
('Segal', 2)
0.01
('.', 2)
0.01


In [98]:
label

2

## 1.2 Hinge Loss

## 1.3 Feature Weight Analysis

In [177]:
d = {'words': [a[0] for a in f_weight_dic.keys()], 'label': [a[1] for a in f_weight_dic.keys()],\
     'weight': f_weight_dic.values()}
f_w_df = pd.DataFrame(d)

In [180]:
f_w_df[f_w_df['label'] == 0].sort_values(by = 'weight', ascending = False).head(10)

,label,weight,words
10624,0,0.03,punches
13143,0,0.03,lacks
23338,0,0.03,amoral
28358,0,0.03,matter
3730,0,0.03,a
4342,0,0.03,wartime
14333,0,0.02,slavery
5594,0,0.02,institutionalized
24424,0,0.02,spook-a-rama
10175,0,0.02,befallen


In [181]:
f_w_df[f_w_df['label'] == 1].sort_values(by = 'weight', ascending = False).head(10)

,label,weight,words
34863,1,0.16,hopeful
29667,1,0.12,band
2572,1,0.12,son
11356,1,0.10,China
9552,1,0.10,producer
30525,1,0.10,recognizable
22385,1,0.09,relief
31152,1,0.09,eyes
16067,1,0.09,post
17446,1,0.09,punches


In [182]:
f_w_df[f_w_df['label'] == 2].sort_values(by = 'weight', ascending = False).head(10)

,label,weight,words
2313,2,112.94,and
7156,2,92.48,","
22210,2,81.65,the
22946,2,79.84,a
30671,2,71.43,of
18262,2,44.56,.
5552,2,33.39,'s
2661,2,32.69,with
18670,2,28.00,film
15597,2,26.59,that


## 1.4 Error Analysis 

## 1.5 Feature Engineering

### 1.5.1 New feature template 1: lowercase words, remove punctuation and lemmatize text

In [194]:
train = pd.read_table('sst3.train', header = None)
train.columns = ['text', 'label']

dev = pd.read_table('sst3.dev', header = None)
dev.columns = ['text', 'label']

devtest = pd.read_table('sst3.devtest', header = None)
devtest.columns = ['text', 'label'] 


def process_1(t):
    l = t.lower().translate(string.maketrans("",""), string.punctuation).strip().split()
    temp = [WordNetLemmatizer().lemmatize(x) for x in l if x != '' and x not in stopwords]

    return temp

def process_df_1(df):
    df['tokens'] = df['text'].apply(lambda x: process_1(x), 1)
    df.reset_index(inplace = True)
#     return df

In [195]:
process_df_1(train)
process_df_1(dev)
process_df_1(devtest)

NameError: global name 'stopwords' is not defined